In [1]:
import os
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from joblib import Parallel, delayed, cpu_count
from tqdm import tqdm  # <-- Added for progress bar

from src.env import Env
from src.agents import RB, MPC
from src.utils import (load_weather_data, load_armax_model)
from src.flex import envelope_for_zone_day
#os.chdir('../'); print(os.getcwd())
os.chdir('/Users/edouardpaupe/Desktop/magnify-main_DATABASE')

HORIZON_HOURS = 24
STEPS_PER_HOUR = 4
CLIMATE_IDS = range(6)  # 0–5

## Climate Data


In [ ]:
# Load the climate data
path = os.path.join('armax_models','archetypes', 'meteo', 'pop_weighted')
weather_df = load_weather_data(file_path=path, climate_id=0) #returns a dataframe with weather data (timestamp, T_amb in degrees, solar irradiance in W/m2)
#climate_id corresponds to the climate scenario of the building, going from 0 to 5.
weather_df.head()

In [ ]:
def save_weather_daily_chunks(path, output_root="input_features/climate_scenarios", climate_ids=range(6)):
    """
    Load weather data for each climate scenario (0..5), split into daily 96-step chunks,
    and save CSV files under input_features/climate_scenarios/climate_{climate_id}.

    The period is restricted to [2020-01-01 00:00:00, 2020-12-30 23:00:00] with 15-min resolution.
    """
    os.makedirs(output_root, exist_ok=True)
    start = pd.Timestamp("2020-01-01 00:00:00")
    end = pd.Timestamp("2020-12-30 23:00:00")

    for cid in climate_ids:
        # Load and restrict to requested window
        df = load_weather_data(file_path=path, climate_id=cid).sort_index() # Ensure datetime index is sorted
        df = df.loc[start:end]

        # Create per-climate output folder
        out_dir = os.path.join(output_root, f"climate_{cid}") 
        os.makedirs(out_dir, exist_ok=True)

        # Iterate over days in the window
        for day_start in pd.date_range(start=start.normalize(), end=end.normalize(), freq="D"):
            day_end = day_start + pd.Timedelta(days=1)
            day_df = df.loc[(df.index >= day_start) & (df.index < day_end)]

            # Expect 96 rows (15-min × 24h); skip incomplete days
            if len(day_df) != 96:
                print(f"[climate {cid}] Skipping {day_start.date()} (found {len(day_df)} rows, expected 96).")
                continue

            # Keep columns and save
            if "T_amb" in day_df.columns and "irrad" in day_df.columns:
                save_df = day_df[["T_amb", "irrad"]].copy()
            else:
                save_df = day_df.copy()

            save_df.index.name = "time"
            day, month, year = day_start.day, day_start.month, day_start.year
            fname = f"climate{cid}_{year}_{month}_{day}.csv"
            fpath = os.path.join(out_dir, fname)

            if os.path.exists(fpath):
                # Skip existing
                continue

            save_df.reset_index().to_csv(fpath, index=False)

        print(f"[climate {cid}] Daily CSVs saved in: {out_dir}")


path = os.path.join('armax_models', 'archetypes', 'meteo', 'pop_weighted')
save_weather_daily_chunks(path)


## Building Archetypes Weights and Biases. 

In [5]:
# Example building folder:

# Create the path to a specific building archetype folder
building_folder = os.path.join('armax_models', 'archetypes', 'ep_SFH_age_0_climate_0_1241')  # 'armax_models/archetypes/ep_SFH_age_0_climate_0_649'

# Load the ARMAX model configuration for this building
armax_config = load_armax_model(
    building_folder_path=building_folder,  # path to the building folder
    steps_per_hour=4,                     # time discretization (4 = 15min steps)
)
#load_armax_model returns a dictionary with the ARMAX model configuration

print(armax_config)

{'armax_coef': array([[8.65e-01, 0.00e+00, 2.15e-02, 0.00e+00, 9.19e-02],
       [4.47e-03, 8.98e-01, 1.24e-02, 2.73e-03, 7.97e-02],
       [3.96e-03, 0.00e+00, 8.67e-01, 0.00e+00, 3.04e-02],
       [5.08e-03, 0.00e+00, 0.00e+00, 8.87e-01, 3.83e-02],
       [2.91e-02, 3.41e-02, 2.16e-02, 3.89e-02, 7.17e-01],
       [2.47e-02, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00],
       [0.00e+00, 8.95e-03, 0.00e+00, 7.62e-03, 0.00e+00],
       [4.64e-04, 1.95e-03, 2.20e-03, 2.22e-04, 6.93e-04],
       [8.52e-05, 3.47e-04, 2.64e-04, 0.00e+00, 0.00e+00],
       [5.49e-05, 2.45e-04, 7.43e-04, 1.56e-06, 5.65e-05],
       [4.05e-05, 1.95e-04, 8.20e-04, 3.06e-04, 7.22e-05],
       [7.45e-05, 1.67e-04, 7.35e-04, 4.07e-04, 3.88e-05],
       [0.00e+00, 1.08e-03, 0.00e+00, 0.00e+00, 0.00e+00],
       [0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00],
       [0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00],
       [5.21e-01, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00],
       [0.00e+00, 5.26e-01, 0.00e+00, 0.0

In [6]:
print(armax_config['armax_coef'])

[[8.65e-01 0.00e+00 2.15e-02 0.00e+00 9.19e-02]
 [4.47e-03 8.98e-01 1.24e-02 2.73e-03 7.97e-02]
 [3.96e-03 0.00e+00 8.67e-01 0.00e+00 3.04e-02]
 [5.08e-03 0.00e+00 0.00e+00 8.87e-01 3.83e-02]
 [2.91e-02 3.41e-02 2.16e-02 3.89e-02 7.17e-01]
 [2.47e-02 0.00e+00 0.00e+00 0.00e+00 0.00e+00]
 [0.00e+00 8.95e-03 0.00e+00 7.62e-03 0.00e+00]
 [4.64e-04 1.95e-03 2.20e-03 2.22e-04 6.93e-04]
 [8.52e-05 3.47e-04 2.64e-04 0.00e+00 0.00e+00]
 [5.49e-05 2.45e-04 7.43e-04 1.56e-06 5.65e-05]
 [4.05e-05 1.95e-04 8.20e-04 3.06e-04 7.22e-05]
 [7.45e-05 1.67e-04 7.35e-04 4.07e-04 3.88e-05]
 [0.00e+00 1.08e-03 0.00e+00 0.00e+00 0.00e+00]
 [0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00]
 [0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00]
 [5.21e-01 0.00e+00 0.00e+00 0.00e+00 0.00e+00]
 [0.00e+00 5.26e-01 0.00e+00 0.00e+00 0.00e+00]
 [0.00e+00 0.00e+00 5.09e-01 0.00e+00 0.00e+00]
 [0.00e+00 0.00e+00 0.00e+00 5.41e-01 0.00e+00]
 [0.00e+00 0.00e+00 0.00e+00 0.00e+00 1.59e-01]
 [0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.

In [7]:
print(armax_config['armax_intercept'])

[[1.34 ]
 [0.975]
 [1.24 ]
 [1.03 ]
 [0.727]]


In [8]:
BUILDING_IDS = [
    "ep_SFH_age_0_climate_0_649", "ep_SFH_age_0_climate_0_821", "ep_SFH_age_0_climate_0_1241",
    "ep_SFH_age_0_climate_1_259", "ep_SFH_age_0_climate_1_493", "ep_SFH_age_0_climate_1_535",
    "ep_SFH_age_0_climate_2_1325", "ep_SFH_age_0_climate_2_1691", "ep_SFH_age_0_climate_2_1972",
    "ep_SFH_age_0_climate_3_955", "ep_SFH_age_0_climate_3_1081", "ep_SFH_age_0_climate_3_1123",
    "ep_SFH_age_0_climate_4_1072", "ep_SFH_age_0_climate_4_1688", "ep_SFH_age_0_climate_4_1709",
    "ep_SFH_age_0_climate_5_417", "ep_SFH_age_0_climate_5_758", "ep_SFH_age_0_climate_5_928",
    "ep_SFH_age_1_climate_0_42", "ep_SFH_age_1_climate_0_168", "ep_SFH_age_1_climate_0_249",
    "ep_SFH_age_1_climate_1_32", "ep_SFH_age_1_climate_1_429", "ep_SFH_age_1_climate_1_458",
    "ep_SFH_age_1_climate_2_762", "ep_SFH_age_1_climate_2_852", "ep_SFH_age_1_climate_2_1161",
    "ep_SFH_age_1_climate_3_260", "ep_SFH_age_1_climate_3_451", "ep_SFH_age_1_climate_3_597"
]

# Create output directories
weights_dir = os.path.join('input_features', 'weights')
biases_dir = os.path.join('input_features', 'biases')
os.makedirs(weights_dir, exist_ok=True)
os.makedirs(biases_dir, exist_ok=True)

# Base path for building archetypes
base_path = os.path.join('armax_models', 'archetypes')

for building_id in BUILDING_IDS:
    # Extract building number (last 3 or 4 digits)
    building_num = building_id.split('_')[-1]
    
    # Full path to building folder
    building_folder = os.path.join(base_path, building_id)
    
    try:
        # Load ARMAX model configuration
        armax_config = load_armax_model(
            building_folder_path=building_folder,
            steps_per_hour=4
        )
        
        # Extract and save weights (armax_coef)
        weights = armax_config['armax_coef']
        weights_df = pd.DataFrame(weights)
        weights_path = os.path.join(weights_dir, f'weights_build{building_num}.csv')
        
        if not os.path.exists(weights_path):
            weights_df.to_csv(weights_path, index=False)
            print(f"✓ Saved weights for building {building_num}")
        else:
            print(f"⊘ Weights file already exists for building {building_num}, skipping")
        
        # Extract and save biases (armax_intercept)
        biases = armax_config['armax_intercept']
        biases_df = pd.DataFrame(biases)
        biases_path = os.path.join(biases_dir, f'biases_build{building_num}.csv')
        
        if not os.path.exists(biases_path):
            biases_df.to_csv(biases_path, index=False)
            print(f"✓ Saved biases for building {building_num}")
        else:
            print(f"⊘ Biases file already exists for building {building_num}, skipping")
            
    except Exception as e:
        print(f"✗ Error processing building {building_id}: {e}")

print("\n=== Processing complete ===")
print(f"Weights saved in: {weights_dir}")
print(f"Biases saved in: {biases_dir}")

✓ Saved weights for building 649
✓ Saved biases for building 649
✓ Saved weights for building 821
✓ Saved biases for building 821
✓ Saved weights for building 1241
✓ Saved biases for building 1241
✓ Saved weights for building 259
✓ Saved biases for building 259
✓ Saved weights for building 493
✓ Saved biases for building 493
✓ Saved weights for building 535
✓ Saved biases for building 535
✓ Saved weights for building 1325
✓ Saved biases for building 1325
✓ Saved weights for building 1691
✓ Saved biases for building 1691
✓ Saved weights for building 1972
✓ Saved biases for building 1972
✓ Saved weights for building 955
✓ Saved biases for building 955
✓ Saved weights for building 1081
✓ Saved biases for building 1081
✓ Saved weights for building 1123
✓ Saved biases for building 1123
✓ Saved weights for building 1072
✓ Saved biases for building 1072
✓ Saved weights for building 1688
✓ Saved biases for building 1688
✓ Saved weights for building 1709
✓ Saved biases for building 1709
✓ Saved